In [1]:
%matplotlib inline
import os
import matplotlib.pyplot as plt
from PIL import Image
import sys
import shutil
from scipy.misc import imsave,imread
import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D, GlobalAveragePooling1D

from keras.layers.normalization import BatchNormalization
from keras import backend as K
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.applications import imagenet_utils
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import h5py
import csv


Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1050 (CNMeM is disabled, cuDNN not available)


In [6]:
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization

In [2]:
# LOADING DATA

def load_images(images_path, path_to_labels):
        path_to_images = images_path
        labels_file = path_to_labels
        images_labels = {}
        with open(labels_file, 'r') as f:
            
            dict_labels = dict([line.strip().split() for line in f.readlines()])
        # List files in this directory
        files = os.listdir(path_to_images)
        
        # Avoid hidden files
        files = filter( lambda files: not files.startswith('.'), files)
        #files = files[0:1000]

        # Create structure for holding images
        images = np.zeros((len(files), 64,64,3), dtype=np.uint8)
        labels = np.zeros(len(files), dtype=np.uint8)
        for fid, file in enumerate(files):
                if fid % 1000 == 0:
                        print(fid)
                image = imread(path_to_images + '/' + file)
                if image.shape == (64,64):
                    print(file)
                images[fid] = image
                labels[fid] = int(dict_labels[file])
        return images, labels, files
    
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [3]:
def conv_dense(n_channels = 3, n_features = 256, reg = 1e-5, init = 'he_normal'):
        model_n = Sequential()

        model_n.add(Conv2D(32, (3, 3),input_shape=(64, 64, 3),padding='same'))
        model_n.add(Activation('relu'))
        model_n.add(Conv2D(32, (3, 3)))
        model_n.add(Activation('relu'))
        model_n.add(MaxPooling2D(pool_size=(2, 2)))
        model_n.add(BatchNormalization())
        model_n.add(Dropout(0.25))


        model_n.add(Conv2D(64, (3, 3), padding='same'))
        model_n.add(Activation('relu'))
        model_n.add(Conv2D(64, (3, 3)))
        model_n.add(Activation('relu'))
        model_n.add(MaxPooling2D(pool_size=(2, 2)))
        model_n.add(BatchNormalization())
        model_n.add(Dropout(0.25))

        model_n.add(Conv2D(64, (3, 3), padding='same'))
        model_n.add(Activation('relu'))
        model_n.add(Conv2D(64, (3, 3)))
        model_n.add(Activation('relu'))
        model_n.add(MaxPooling2D(pool_size=(2, 2)))
        model_n.add(BatchNormalization())
        model_n.add(Dropout(0.25))

        model_n.add(Conv2D(128, (3, 3), padding='same'))
        model_n.add(Activation('relu'))
        model_n.add(Conv2D(128, (3)))
        model_n.add(Activation('relu'))
        model_n.add(MaxPooling2D(pool_size=(2, 2)))
        model_n.add(BatchNormalization())
        model_n.add(Dropout(0.5))

        #model.add(Flatten()) global average pooling
        model_n.add(GlobalAveragePooling2D())
        model_n.add(Dense(512))
        model_n.add(Activation('relu'))
        model_n.add(Dropout(0.5))
        model_n.add(Dense(256))
        model_n.add(Activation('softmax'))

        model_n.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        return model_n

In [4]:
print "Loading Train data..."
x_train, y_train, train_files_name = load_images("./train_resized/",
                                                 "./train_labels.txt")
print 'Loading Test data...'
x_test, a, b = load_images('./test_resized/',
                    './test_labels.txt')

print 'Loading Validation data...'
x_val, y_val, j = load_images('./validation_resized/',
                    './validation_labels.txt')


x_train, y_train = unison_shuffled_copies(x_train, y_train)
#y_train = np_utils.to_categorical(y_train)
y_train -=1
y_val -=1

Loading Train data...
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
Loading Test data...
0
1000
2000
3000
4000
5000
6000
7000
Loading Validation data...
0
1000
2000


In [5]:
#y_train = np_utils.to_categorical(y_train, 256)
datagen = ImageDataGenerator(
                             width_shift_range=0.05, 
                             height_shift_range=0.05, 
                             horizontal_flip=True
                             
)


In [8]:
n = conv_dense()

#model_checkpoint = ModelCheckpoint(filepath='/home/bohdan/Desktop/Models/model2-{epoch:02d}-{val_loss:.2f}.hdf5',
#                                   monitor='val_loss', verbose=1, save_best_only=True, mode='min')
#n.load_weights('/home/bohdan/Desktop/Models/model2-08-3.97.hdf5')
print(n.summary())
# n.fit(x_train, y_train,
#           batch_size=200,
#           epochs=150,
#           verbose=1,
#           callbacks=[model_checkpoint],
#           #validation_split = 0.2
#           validation_data=(x_val, y_val)
#      )

n.fit_generator(datagen.flow(x_train, y_train, 
                             batch_size=100),
                             steps_per_epoch=200, epochs=10,
                             #callbacks=[model_checkpoint],
                             validation_data = (x_val, y_val)
               )


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
activation_11 (Activation)   (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 62, 62, 32)        9248      
_________________________________________________________________
activation_12 (Activation)   (None, 62, 62, 32)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 31, 31, 32)        128       
_________________________________________________________________
dropout_6 (Dropout)          (None, 31, 31, 32)        0         
__________

In [11]:

n.fit_generator(datagen.flow(x_train, y_train, 
                             batch_size=100),
                             steps_per_epoch=200, epochs=5,
                             #callbacks=[model_checkpoint],
                             validation_data = (x_val, y_val)
               )

Epoch 1/5
200/200 [==============================] - 128s - loss: 2.0747 - acc: 0.4859 - val_loss: 3.8279 - val_acc: 0.2434

In [12]:
predicted_labels = n.predict_classes(x_test)
predicted_labels += 1

7680/7680 [==============================] - 19s     

In [13]:
for i in range(len(predicted_labels)):
    if predicted_labels[i] == 20:
        print b[i]
    

4917.jpg
4357.jpg
16252.jpg
19625.jpg
11688.jpg
26623.jpg
28156.jpg
18978.jpg
9037.jpg
30626.jpg
28056.jpg
18969.jpg
28743.jpg
15027.jpg
24352.jpg
7134.jpg
11769.jpg
4731.jpg
32386.jpg
28903.jpg
7924.jpg
20011.jpg
23056.jpg
26803.jpg
13191.jpg
14924.jpg
3083.jpg
32572.jpg
17829.jpg
8434.jpg
32487.jpg
25636.jpg
15610.jpg


In [14]:
with open('beat30_results.csv', 'w') as csvfile:
    fieldnames = ['image', 'class']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for i in range(len(predicted_labels)):
        writer.writerow({'image': b[i], 'class': predicted_labels[i]})